In [4]:
# Instala o Google Chrome
!apt-get update -qq > /dev/null
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" | tee /etc/apt/sources.list.d/google-chrome.list
!apt-get update -qq > /dev/null
!apt-get install -qq google-chrome-stable

# Instala o Selenium e o webdriver-manager
!pip install -U selenium webdriver-manager

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
OK
deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main
W: http://dl.google.com/linux/chrome/deb/dists/stable/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libvulkan1:amd64.
(Reading database ... 123623 files and directories currently installed.)
Preparing to unpack .../libvulkan1_1.3.204.1-2_amd64.deb ...
Unpacking libvulkan1:amd64 (1.3.204.1-2) ...
Selecting previously unselected package google-chrome-stable.
Preparing to unpack .../google-chrome-stable_130.0.6723.116-1_am

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pickle
import os

# Configuração do WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless=new")  # Executa o navegador em modo headless (opcional)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# URL da página de resultados da pesquisa
url = "https://www.linkedin.com/jobs/search/?currentJobId=4046225014&geoId=106057199&keywords=%22machine%20learning%22&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true"

li_at_cookie = "AQEDATI8OtEATpypAAABkwM9s_0AAAGTJ0o3_U0AMKKTtJsrrHcVgIfTGU4wQWtYpKaes0szHuOLUZDZdhRSFk1MGzn9g4PQwUFhACSi4dpmCUm7XzcZKbCMxPIP_cZ7nzJTAB2KXlqFMR0uv0OUb1-7"
driver.get("https://www.linkedin.com") # Navegue para o LinkedIn primeiro

driver.add_cookie({"name": "li_at", "value": li_at_cookie, "domain": ".linkedin.com"})
# Adicione outros cookies, se necessário, usando o mesmo formato

# Acessa a página
driver.get(url)

# Aguarda o carregamento da página (ajuste o tempo conforme necessário)
time.sleep(5)  # Aumentado o tempo de espera

# Encontra todos os cards de vagas
job_cards = driver.find_elements(By.CLASS_NAME, "job-card-container--clickable")

# Abre cada card e extrai a descrição
all_descriptions = []
for job_card in job_cards:
    job_card.click()  # Clica no card da vaga
    time.sleep(5)  # Aguarda o carregamento da descrição

    # Encontra a descrição da vaga dentro do card aberto
    job_description = driver.find_element(By.XPATH, '//article[contains(@class, "jobs-description__container")]')
    all_descriptions.append(job_description.text)  # Adiciona a descrição à lista

# Salva todas as descrições em um arquivo
with open('job_descriptions.txt', 'w', encoding='utf-8') as f:
    for description in all_descriptions:
        f.write(description + '\n')

print(f"Descrições de {len(all_descriptions)} empregos extraídas e salvas em 'job_descriptions.txt'")

# # Tenta encontrar as descrições dos empregos
# job_descriptions = driver.find_elements(By.XPATH, '//article[contains(@class, "jobs-description__container")]')

# # Verifica se foram encontradas descrições
# if job_descriptions:
#     # Extrai o texto das descrições e salva em uma lista
#     descriptions = [job.text for job in job_descriptions]

#     # Salva as descrições em um arquivo
#     with open('job_descriptions.txt', 'w', encoding='utf-8') as f:
#         for description in descriptions:
#             f.write(description + '\n')

#     print("Descrições de empregos extraídas e salvas em 'job_descriptions.txt'")
# else:
#     print("Nenhuma descrição de emprego encontrada.")
#     # Salva o HTML da página para depuração
#     with open("page_source.html", "w", encoding="utf-8") as f:
#         f.write(driver.page_source)

#     # Salva um print da tela para depuração
#     driver.save_screenshot("page_screenshot.png")
#     print("HTML da página salvo em 'page_source.html'")
#     print("Print da tela salvo em 'page_screenshot.png'")

# Fecha o navegador
driver.quit()

Descrições de 7 empregos extraídas e salvas em 'job_descriptions.txt'


In [8]:
!ls

job_descriptions.txt  page_screenshot.png  page_source.html  sample_data


In [ ]:
# import os

# os.environ['LINKEDIN_EMAIL'] = 'gacarfaria@gmail.com'
# os.environ['LINKEDIN_SENHA'] = 'G4c4rf4r1421#'